In [ ]:
import warnings
warnings.filterwarnings('ignore')

import random
import time
import multiprocessing as mp
import numpy as np

import mxnet as mx
from mxnet import nd, gluon, autograd

import gluonnlp as nlp
import pickle

random.seed(123)
np.random.seed(123)
mx.random.seed(123)

In [103]:
train_dataset = pickle.load(open('../data/dev_processed.p', 'rb'))
test_dataset = pickle.load(open('../data/dev_processed.p', 'rb'))

In [104]:
vocabulary = {'<pad>': [0, 1], '<unk>': [1, 1], '<BOS>': [2, 1], '<EOS>': [3, 1]}
for item in train_dataset + test_dataset:
    words = item[2].split(' ')
    for word in words:
        if word in vocabulary:
            vocabulary[word][1] += 1
        else:
            vocabulary[word] = [len(vocabulary), 1]

vocabulary_inv = {}
for key in vocabulary:
    vocabulary_inv[vocabulary[key][0]] = key

In [4]:
def preprocess(x):
    name, audio, words = x
    split_words = ['<BOS>'] + words.split(' ') + ['<EOS>']
    return audio, np.array([vocabulary[word][0] for word in split_words]), float(len(audio)), float(len(split_words))

def get_length(x):
    return float(len(x[1]))

def preprocess_dataset(dataset):
    start = time.time()
    with mp.Pool() as pool:
        dataset = gluon.data.SimpleDataset(pool.map(preprocess, dataset))
        lengths = gluon.data.SimpleDataset(pool.map(get_length, dataset))
    end = time.time()
    print('Done! Processing Time={:.2f}s, #Samples={}'.format(end - start, len(dataset)))
    return dataset, lengths

In [5]:
train_dataset, train_data_lengths = preprocess_dataset(train_dataset)
test_dataset, test_data_lengths = preprocess_dataset(test_dataset)

Done! Processing Time=2.48s, #Samples=2703
Done! Processing Time=1.96s, #Samples=2703


In [6]:
learning_rate, batch_size = 0.005, 32
bucket_num, bucket_ratio = 10, 0.2
grad_clip = None
log_interval = 5

def get_dataloader():
    batchify_fn = nlp.data.batchify.Tuple(
        nlp.data.batchify.Pad(dtype='float32'),
        nlp.data.batchify.Pad(dtype='float32'),
        nlp.data.batchify.Stack(dtype='float32'),
        nlp.data.batchify.Stack(dtype='float32'))
    batch_sampler = nlp.data.sampler.FixedBucketSampler(
        train_data_lengths,
        batch_size=batch_size,
        num_buckets=bucket_num,
        ratio=bucket_ratio,
        shuffle=True)
    print(batch_sampler.stats())

    train_dataloader = gluon.data.DataLoader(
        dataset=train_dataset,
        batch_sampler=batch_sampler,
        batchify_fn=batchify_fn)
    test_dataloader = gluon.data.DataLoader(
        dataset=test_dataset,
        batch_size=batch_size,
        shuffle=False,
        batchify_fn=batchify_fn)
    return train_dataloader, test_dataloader

train_dataloader, test_dataloader = get_dataloader()

FixedBucketSampler:
  sample_num=2703, batch_num=82
  key=[13, 22, 31, 40, 49, 58, 67, 76, 85, 94]
  cnt=[837, 805, 531, 282, 120, 70, 30, 15, 8, 5]
  batch_size=[46, 32, 32, 32, 32, 32, 32, 32, 32, 32]


In [7]:
context = mx.cpu()

In [ ]:
for i, example in enumerate(train_dataloader):
    if i >= 1:
        break
    print(example)

In [9]:
import numpy as np
import mxnet as mx
from io import open
from mxnet import gluon, autograd
from mxnet.gluon import nn, rnn, Block
from mxnet import ndarray as F

class SubSampler(gluon.HybridBlock):
    def __init__(self, size=3, prefix=None, params=None):
        super(SubSampler, self).__init__(prefix=prefix, params=params)
        self.size = size

    def forward(self, data, valid_length):
        masked_encoded = F.SequenceMask(data,
                                        sequence_length=valid_length,
                                        use_sequence_length=True)
        subsampled = F.Pooling(masked_encoded.swapaxes(0,2), kernel=(self.size), pool_type='max', stride=self.size).swapaxes(0,2)
        sub_valid_length = mx.nd.ceil(valid_length / self.size)
        return subsampled, sub_valid_length

In [191]:
class EncoderRNN(Block):
    def __init__(self, input_size, hidden_size):
        super(EncoderRNN, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size

        with self.name_scope():
            self.rnn1 = rnn.GRU(hidden_size, input_size=self.input_size)
            self.subsampler = SubSampler(size=3)
            self.rnn2 = rnn.GRU(hidden_size, input_size=self.hidden_size)

    def forward(self, input, hidden, lengths):
        input = input.swapaxes(0, 1)
        output, hidden1 = self.rnn1(input, hidden)
        subsampled, sub_lengths = self.subsampler(output, lengths)
        output, hidden2 = self.rnn2(subsampled, hidden)
        return output, hidden2, sub_lengths

    def initHidden(self, batchsize, ctx):
        return [F.zeros((1, batchsize, self.hidden_size), ctx=ctx)]

In [192]:
class DecoderRNN(Block):
    def __init__(self, output_size, hidden_size):
        super(DecoderRNN, self).__init__()
        self.output_size = output_size
        self.hidden_size = hidden_size

        with self.name_scope():
            self.embedding = nn.Embedding(output_size, hidden_size)
            self.rnn = rnn.GRU(hidden_size, input_size=self.hidden_size)
            self.out = nn.Dense(output_size, in_units=self.hidden_size, flatten=False)

    def forward(self, input, hidden):
        output = self.embedding(input)
        output, hidden = self.rnn(output, [hidden[0].swapaxes(0, 1)])
        output = self.out(output)
        return output, hidden

    def initHidden(self, batchsize, ctx):
        return [F.zeros((1, batchsize, self.hidden_size), ctx=ctx)]

In [193]:
class Seq2Seq(Block):
    def __init__(self, input_size, output_size, hidden_size):
        super(Seq2Seq, self).__init__()
        self.input_size = input_size
        self.output_size = output_size
        self.hidden_size = hidden_size
        with self.name_scope():
            self.encoder = EncoderRNN(input_size=input_size, hidden_size=hidden_size)
            self.decoder = DecoderRNN(hidden_size=hidden_size, output_size=output_size)
    
    def forward(self, input, hidden, lengths, words):
        encoder_outputs, encoder_hidden, encoder_out_lengths = self.encoder(input, hidden, lengths)
        r_enc_lens = mx.ndarray.repeat(encoder_out_lengths.expand_dims(1), repeats=encoder_outputs.shape[2], axis=1)
        encoder_final_states = mx.ndarray.pick(encoder_outputs.swapaxes(0,1), 
                                               r_enc_lens,
                                               axis=1).expand_dims(0)
        # CURRENT: Get the last hidden state according to the lengths
        # TODO   : Get the attended state
        decoder_hidden = [encoder_final_states.swapaxes(0, 1)]
        decoder_outputs, decoder_hidden = self.decoder(words.swapaxes(0,1), decoder_hidden)
        decoder_outputs = decoder_outputs.swapaxes(0,1)
        return decoder_outputs

In [194]:
from mxnet.gluon.loss import Loss as Loss

class SoftmaxSequenceCrossEntropyLoss(Loss):
    def __init__(self, axis=-1, sparse_label=True, from_logits=False, weight=None,
                 batch_axis=0, **kwargs):
        super(SoftmaxSequenceCrossEntropyLoss, self).__init__(
            weight, batch_axis, **kwargs)
        self._axis = axis
        self._sparse_label = sparse_label
        self._from_logits = from_logits
    
    def hybrid_forward(self, F, pred, label, valid_length):
        if not self._from_logits:
            pred = F.log_softmax(pred, self._axis)
        loss = mx.nd.squeeze(-F.pick(pred, label, axis=self._axis, keepdims=True), axis=2)
        loss = F.SequenceMask(loss.swapaxes(0,1), 
                              sequence_length=valid_length,
                              use_sequence_length=True).swapaxes(0,1)
        return F.mean(loss, axis=self._batch_axis, exclude=True)

In [195]:
net = Seq2Seq(input_size=13, output_size=34798, hidden_size=16)
net.initialize(mx.init.Xavier(), ctx=context)

class beamDecoder(object):
    def __init__(self, model):
        self._model = model
    def __call__(self, inputs, states):
        outputs, states = self._model.decoder(mx.nd.expand_dims(inputs, axis=0), states)
        return outputs[0], [states[0].swapaxes(0,1)]

scorer = nlp.model.BeamSearchScorer(alpha=0, K=5, from_logits=False)
eos_id = vocabulary['<EOS>'][0]
beam_sampler = nlp.model.BeamSearchSampler(beam_size=5,
                                           decoder=beamDecoder(net),
                                           eos_id=eos_id,
                                           scorer=scorer,
                                           max_length=20)

In [234]:
def get_sequence_accuracy(s1, l1, s2, l2):
    s1 = mx.nd.cast(s1, dtype='int32')
    l1 = mx.nd.cast(l1, dtype='int32')
    s2 = mx.nd.cast(s2, dtype='int32')
    l2 = mx.nd.cast(l2, dtype='int32')
    padding = mx.nd.zeros((s1.shape[0], abs(s1.shape[1] - s2.shape[1])), dtype=s2.dtype)
    if s1.shape[1] > s2.shape[1]:
        s2 = mx.nd.concat(s2, padding, dim=1)
    else:
        s1 = mx.nd.concat(s1, padding, dim=1) 
    accs = F.SequenceMask((s1 == s2).swapaxes(0,1), 
                          sequence_length=mx.nd.minimum(l1, l2), 
                          use_sequence_length=True)
    return (mx.nd.cast(accs.sum(), dtype='float32')/mx.nd.cast(l1.sum(), dtype='float32')).asnumpy().item()

In [235]:
def evaluate(net, context):
    for i, (audio, words, alength, wlength) in enumerate(test_dataloader):
        encoder_hidden = net.encoder.initHidden(len(audio), context)
        encoder_outputs, encoder_hidden, encoder_out_lengths = net.encoder(audio.as_in_context(context), encoder_hidden, alength)
        r_enc_lens = mx.ndarray.repeat(encoder_out_lengths.expand_dims(1), repeats=encoder_outputs.shape[2], axis=1)
        encoder_final_states = mx.ndarray.pick(encoder_outputs.swapaxes(0,1), 
                                               r_enc_lens,
                                               axis=1).expand_dims(0)
        decoder_hidden = [encoder_final_states.swapaxes(0, 1)]

        inputs = mx.nd.array([2] * words.shape[0])
        samples, scores, valid_lengths = beam_sampler(inputs, decoder_hidden)
        best_samples = samples[:,0,1:-1]
        best_vlens = valid_lengths[:,0]
        return get_sequence_accuracy(words, wlength, best_samples, best_vlens)

In [238]:
def train(net, context, epochs):
    trainer = gluon.Trainer(net.collect_params(), 'ftml',
                            {'learning_rate': learning_rate})
    loss = SoftmaxSequenceCrossEntropyLoss()

    parameters = net.collect_params().values()

    for epoch in range(epochs):
        start_epoch_time = time.time()
        epoch_L = 0.0
        epoch_sent_num = 0
        epoch_wc = 0

        start_log_interval_time = time.time()
        log_interval_wc = 0
        log_interval_sent_num = 0
        log_interval_L = 0.0

        for i, (audio, words, alength, wlength) in enumerate(train_dataloader):
            wc = alength.sum().asscalar()
            log_interval_wc += wc
            epoch_wc += wc
            log_interval_sent_num += audio.shape[1]
            epoch_sent_num += audio.shape[1]
            with autograd.record():
                encoder_hidden = net.encoder.initHidden(len(audio), context)
                decoder_outputs = net(audio.as_in_context(context), encoder_hidden, alength, words.as_in_context(context))
                L = loss(decoder_outputs, words.as_in_context(context), wlength).sum()
            L.backward()
            
            if grad_clip:
                gluon.utils.clip_global_norm(
                    [p.grad(context) for p in parameters],
                    grad_clip)
            
            trainer.step(1)
            log_interval_L += L.asscalar()
            epoch_L += L.asscalar()
            if (i + 1) % log_interval == 0:
                print(
                    '[Epoch {} Batch {}/{}] elapsed {:.2f} s, '
                    'avg loss {:.6f}, throughput {:.2f}K fps'.format(
                        epoch, i + 1, len(train_dataloader),
                        time.time() - start_log_interval_time,
                        log_interval_L / log_interval_sent_num, log_interval_wc
                        / 1000 / (time.time() - start_log_interval_time)))
                start_log_interval_time = time.time()
                log_interval_wc = 0
                log_interval_sent_num = 0
                log_interval_L = 0
        
        end_epoch_time = time.time()
        test_acc = evaluate(net, context)
        print('[Epoch {}] train avg loss {:.6f}, test acc {:.2f}, '
              'throughput {:.2f}K fps'.format(
              epoch, epoch_L / epoch_sent_num, test_acc,
              epoch_wc / 1000 / (end_epoch_time - start_epoch_time)))

In [ ]:
train(net, context, 1)

In [148]:
def generate_sequences(sampler, inputs, begin_states, num_print_outcomes):
    samples, scores, valid_lengths = sampler(inputs, begin_states)
    print(valid_lengths.shape)
    print('Generation Result:')
    
    for sample_id in range(samples.shape[0]):
        sample = samples[sample_id].asnumpy()
        score = scores[sample_id].asnumpy()
        valid_length = valid_lengths[sample_id].asnumpy()

        for i in range(num_print_outcomes):
            sentence = []
            for ele in sample[i][:valid_length[i]]:
                sentence.append(vocabulary_inv[ele])
            print([' '.join(sentence), score[i]])

In [ ]:
inputs = mx.nd.array([2] * 32)
begin_states = [mx.nd.array([[[0]*16]] * 32)]
generate_sequences(beam_sampler, inputs, begin_states, 1)

In [233]:
a = mx.nd.array([[2,2,2,2], [3,3,3,3]])
al = mx.nd.array([3,4])
b = mx.nd.array([[2,2,1], [2,3,3]])
bl = mx.nd.array([2,3])
get_sequence_accuracy(a,al,b,bl)

0.5714285969734192